In [ ]:
 
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import os
from dotenv import load_dotenv
from huggingface_hub import login
import json

/home/mrosaria/Projects/NLP/GymRat/ratenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_private_keys(key_name):
    load_dotenv()
    key = os.getenv(key_name)
    if not key:
        raise ValueError(f"Missing key for {key_name} in .env") 
    return key

huggingface_hub_token = get_private_keys("huggingface_hub")
login(token=huggingface_hub_token)

In [3]:
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
input_json_file_name = "cleaned_1506_raw_outputs_samples1260_nreps3_new_tok256_chuck_size250_overlap30_.json"
#
# input_json_file_name = "cleaned_1506_raw_outputs_samples1260_nreps3_new_tok256_chuck_size250_overlap30_.json"
# with open(input_json_file_name, "r") as f:
#     loaded_list = json.load(f)

#input_json_file_name = "cleaned_343_qa_outputs_800_batch16_4bit_30072025.json"
# Load the dataset from JSON file
dataset = load_dataset("json", data_files=input_json_file_name)



In [5]:
splits = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset = splits["train"]
test_dataset = splits["test"]

In [6]:
# Merge 'instruction' and 'output' as prompt and target
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    return {"text": prompt}

In [7]:
dataset = dataset.map(format_example)
test_dataset = test_dataset.map(format_example)
print(dataset[1])

{'instruction': "Squeezing your arms tight to your rib cage while engaging your powerful lat muscles and creating a tremendous amount of core stiffness and upper body stability. Next, engage the rest of your upper body by pulling upward against the bar, not moving it from the raised platform yet, but enough to essentially wedge your body against the bar like a trust fall. If you're doing it correctly, you'll see the bar bend upward slightly. To create this tension with your upper body, you need to simultaneously do the opposite with your lower body. This means creating a tremendous amount of tension in your glutes and hamstrings as you drive your feet hard into the ground. Finding the right balance?.", 'output': 'Micah Mariano deadlift while creating a tremendous amount of tension in your glutes and hamstrings while driving your feet hard into the ground.', 'text': "### Instruction:\nSqueezing your arms tight to your rib cage while engaging your powerful lat muscles and creating a trem

In [8]:
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",  # You can also use "fp4"
#     bnb_4bit_compute_dtype="float16"
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     quantization_config=quant_config,
#     trust_remote_code=True
# )


In [9]:
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     load_in_4bit=True,  # Optional: enable quantization to save memory
#     device_map="auto",
#     trust_remote_code=True
# )

# # Apply LoRA
# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     target_modules=["q_proj", "v_proj"],  # These depend on the model
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM
# )

# model = get_peft_model(model, lora_config)

# # Tokenize the dataset
# def tokenize_function(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

# tokenized_dataset = dataset.map(tokenize_function, batched=True)

# # Training setup
# training_args = TrainingArguments(
#     output_dir="./lora-llama-output",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     learning_rate=2e-4,
#     num_train_epochs=3,
#     logging_dir="./logs",
#     fp16=True,
#     save_total_limit=2,
#     logging_steps=10,
#     save_steps=500,
#     report_to="none"
# )

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer,
#     data_collator=data_collator
# )

# # Start training
# trainer.train()
#TrainOutput(global_step=129, training_loss=1.575617376224015, metrics={'train_runtime': 199.0944, 'train_samples_per_second': 5.168, 'train_steps_per_second': 0.648, 'total_flos': 3273745332436992.0, 'train_loss': 1.575617376224015, 'epoch': 3.0})
## new_model_name = "tinyllama-qa-343samples-finetuned-30072025"



In [ ]:
##### Updated after 1st atempt

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  #or "fp4"
    bnb_4bit_compute_dtype="float16"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)

# Apply LoRA
lora_config = LoraConfig(
    #r=8, With small data, a smaller r (e.g., 4) might prevent overfitting
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # These depend on the model
    lora_dropout=0.05, #You could try slightly higher (0.1) if overfitting.
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset_val = test_dataset.map(tokenize_function, batched=True)

# Training setup
training_args = TrainingArguments(
    output_dir="./lora-llama-output",
    #per_device_train_batch_size=2 with gradient_accumulation_steps=4 is fine (effective batch size 8).
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    #learning_rate=2e-4, 2e-4 is a bit high for fine-tuning on a small dataset. It can cause unstable training or forgetting.
    learning_rate=1e-4,#1e-5,
    #num_train_epochs=3, #With only ~300 samples, 3 epochs might be too little.
    num_train_epochs=10,
    logging_dir="./logs",
    fp16=True,
    save_total_limit=2,
    logging_steps=10,
    save_steps=500,
    report_to="none", 
    #Add weight decay, Helps regularize, reduce overfitting.
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # Used train data also as eval
    eval_dataset=tokenized_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()

#old  TrainOutput(global_step=129, training_loss=1.575617376224015, metrics={'train_runtime': 199.0944, 'train_samples_per_second': 5.168, 'train_steps_per_second': 0.648, 'total_flos': 3273745332436992.0, 'train_loss': 1.575617376224015, 'epoch': 3.0})
#this TrainOutput(global_step=860, training_loss=1.7047795207001442, metrics={'train_runtime': 1317.3933, 'train_samples_per_second': 5.207, 'train_steps_per_second': 0.653, 'total_flos': 2.181310005116928e+16, 'train_loss': 1.7047795207001442, 'epoch': 20.0})
#Step	Training Loss	Validation Loss
# 100	1.906100	1.956670
# 200	1.908100	1.838620
# 300	1.737400	1.735574
# 400	1.606700	1.687118
# 500	1.622300	1.645923
# 600	1.599600	1.621207
# 700	1.585900	1.607686
# 800	1.379700	1.601775
#new_model_name = "tinyllama-v2-qa-343samples-finetuned-31072025"


#Current with 1230 samples
#TrainOutput(global_step=1425, training_loss=1.6043405378910534, metrics={'train_runtime': 4848.0399, 'train_samples_per_second': 4.66, 'train_steps_per_second': 0.294, 'total_flos': 7.183060206354432e+16, 'train_loss': 1.6043405378910534, 'epoch': 15.0})

#with 1230 samples, minus eval 10%, 10 epochs, r=8,  learning_rate=1e-4 08/08/2025
#TrainOutput(global_step=850, training_loss=1.421143362381879, metrics={'train_runtime': 2160.1019, 'train_samples_per_second': 6.273, 'train_steps_per_second': 0.393, 'total_flos': 4.31090857672704e+16, 'train_loss': 1.421143362381879, 'epoch': 10.0})
# Step	Training Loss	Validation Loss
# 100	1.476200	1.503343
# 200	1.467700	1.439405
# 300	1.407400	1.426077
# 400	1.362700	1.420144
# 500	1.359200	1.415425
# 600	1.478300	1.412378
# 700	1.325800	1.412556
# 800	1.381600	1.411611

Map: 100%|██████████| 151/151 [00:00<00:00, 4155.94 examples/s]
/tmp/ipykernel_979850/1998495725.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
100,1.476200,1.503343
200,1.467700,1.439405
300,1.407400,1.426077
400,1.362700,1.420144
500,1.359200,1.415425
600,1.478300,1.412378
700,1.325800,1.412556
800,1.381600,1.411611


TrainOutput(global_step=850, training_loss=1.421143362381879, metrics={'train_runtime': 2160.1019, 'train_samples_per_second': 6.273, 'train_steps_per_second': 0.393, 'total_flos': 4.31090857672704e+16, 'train_loss': 1.421143362381879, 'epoch': 10.0})

In [15]:
TrainOutput

NameError: name 'TrainOutput' is not defined

In [12]:
# new_model_name = "tinyllama-qa-343samples-finetuned-30072025"
new_model_name = "tinyllama-v2-qa-343samples-finetuned-08082025"

trainer.save_model(new_model_name)
tokenizer.save_pretrained(new_model_name)


('tinyllama-v2-qa-343samples-finetuned-08082025/tokenizer_config.json',
 'tinyllama-v2-qa-343samples-finetuned-08082025/special_tokens_map.json',
 'tinyllama-v2-qa-343samples-finetuned-08082025/chat_template.jinja',
 'tinyllama-v2-qa-343samples-finetuned-08082025/tokenizer.json')

# Test NEW model

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
from transformers import pipeline

In [26]:
#Load the Base (Unmodified) Model
base_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")


In [27]:
new_model_name

'tinyllama-v2-qa-343samples-finetuned-04082025'

In [28]:
#Load MY Fine-Tuned LoRA Model
peft_model_dir = f"{new_model_name}"  # Your trained adapter folder
config = PeftConfig.from_pretrained(peft_model_dir)

# Load base model and apply LoRA weights
finetuned_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
finetuned_model = PeftModel.from_pretrained(finetuned_model, peft_model_dir)

finetuned_model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [29]:
from IPython.display import Markdown, display


In [30]:

message = "What is the importance of proper recovery in powerlifting?"
prompt = [{"role": "user", "content": message}]

formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

# Base model generation
base_pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer)
base_output = base_pipe(formatted_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)[0]["generated_text"]

# Fine-tuned model generation
finetuned_pipe = pipeline("text-generation", model=finetuned_model, tokenizer=tokenizer)
finetuned_output = finetuned_pipe(formatted_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)[0]["generated_text"]

# Compare
print("🔹 Base Model Output:\n", base_output)
print("\n🔸 Fine-Tuned Model Output:\n", finetuned_output)


Device set to use cuda:0


Device set to use cuda:0


🔹 Base Model Output:
 <|user|>
What is the importance of proper recovery in powerlifting?</s>
<|assistant|>
Proper recovery is crucial for powerlifters because it allows them to rebuild and repair their muscles, improve their strength, and prepare for the next set of lifts. Proper recovery can help to reduce muscle soreness, improve blood flow to the muscles, and promote muscle growth and repair. Proper recovery also helps to prevent injury by ensuring that the body is fully recovered from the previous set of lifts. Recovery is particularly important for powerlifters who perform repeated sets of heavy lifts over long periods. Without proper recovery, the body may become fatigued and prone to injury. Overall, proper recovery is an essential part of powerlifting and plays a significant role in achieving maximum strength and performance.

🔸 Fine-Tuned Model Output:
 <|user|>
What is the importance of proper recovery in powerlifting?</s>
<|assistant|>
Proper recovery is crucial in powerlif

In [31]:
message = "how to screen for understanding shoulder pain?"
#"What is the 'no pain, no gain' mantra?"
#"What is the importance of proper recovery in powerlifting?"
prompt = [{"role": "user", "content": message}]

formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)


outputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")  # or "cpu"
response_base_model = base_model.generate(
    **outputs,
    max_new_tokens=256,
    do_sample=True,         # enables sampling for variability
    top_p=0.9,               # nucleus sampling
    temperature=0.7,         # randomness control
    pad_token_id=tokenizer.eos_token_id  # avoid pad token issues
)
decoded_base_model = tokenizer.decode(response_base_model[0], skip_special_tokens=True)
print(decoded_base_model)


<|user|>
how to screen for understanding shoulder pain? 
<|assistant|>
To screen for understanding shoulder pain, the following steps can be followed:

1. Perform a thorough examination of the shoulder joint, including a thorough evaluation of shoulder range of motion, shoulder movement patterns, and shoulder pain.

2. Ask the patient to perform a shoulder range of motion exercise, such as bending the arm and rotating the arm, to assess the patient's shoulder mobility.

3. Evaluate the patient's shoulder pain by asking about any specific symptoms or areas of pain, such as a lump or a clicking sound in the shoulder joint.

4. Assess the patient's ability to perform daily activities, such as brushing their teeth, using a cup, or turning a chair, to determine if the shoulder pain is impacting their daily functioning.

5. Ask the patient about any changes in their shoulder function or range of motion over the past few weeks, and consider any changes in their shoulder activities, such as ch

In [32]:
outputs_finetunned = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")  # or "cpu"

response_finetunned_model = finetuned_model.generate(
    **outputs_finetunned,
    max_new_tokens=256,
    do_sample=True,         # enables sampling for variability
    top_p=0.9,               # nucleus sampling
    temperature=0.7,         # randomness control
    pad_token_id=tokenizer.eos_token_id  # avoid pad token issues
)
decoded_finetunned_model = tokenizer.decode(response_finetunned_model[0], skip_special_tokens=True)
print(decoded_finetunned_model)


<|user|>
how to screen for understanding shoulder pain? 
<|assistant|>
To screen for understanding shoulder pain, you can use the following techniques:

1. Neck Mobility Test: Stand with your back against a wall and hold your head in a neutral position. Slowly raise your head and neck until you feel a tightness or pain in your neck. If you feel pain, this is an indication that your shoulder may be experiencing shoulder pain.

2. Shoulder Raising Test: Stand with your back against a wall and hold your arms out to the sides. Slowly raise your arms until you feel a tightness or pain in your shoulder. If you feel pain, this is an indication that your shoulder may be experiencing shoulder pain.

3. Pain Test: Place your hand on your shoulder and feel for any pain or discomfort. If you feel any pain or discomfort, this is an indication that your shoulder may be experiencing shoulder pain.

4. Range of Motion Test: Stand with your back against a wall and hold your arms out to the sides. Slowl

In [33]:
#pprint(decoded_finetunned_model.split('<|assistant|>')[-1])
display(Markdown(decoded_finetunned_model.split('<|assistant|>')[-1]))


To screen for understanding shoulder pain, you can use the following techniques:

1. Neck Mobility Test: Stand with your back against a wall and hold your head in a neutral position. Slowly raise your head and neck until you feel a tightness or pain in your neck. If you feel pain, this is an indication that your shoulder may be experiencing shoulder pain.

2. Shoulder Raising Test: Stand with your back against a wall and hold your arms out to the sides. Slowly raise your arms until you feel a tightness or pain in your shoulder. If you feel pain, this is an indication that your shoulder may be experiencing shoulder pain.

3. Pain Test: Place your hand on your shoulder and feel for any pain or discomfort. If you feel any pain or discomfort, this is an indication that your shoulder may be experiencing shoulder pain.

4. Range of Motion Test: Stand with your back against a wall and hold your arms out to the sides. Slowly move your arms out to the sides until you feel a tightness or pain in your shoulder. If you feel pain, this is an indication that your shoulder

In [34]:
#pprint(decoded_base_model.split('<|assistant|>')[-1])
display(Markdown(decoded_base_model.split('<|assistant|>')[-1]))


To screen for understanding shoulder pain, the following steps can be followed:

1. Perform a thorough examination of the shoulder joint, including a thorough evaluation of shoulder range of motion, shoulder movement patterns, and shoulder pain.

2. Ask the patient to perform a shoulder range of motion exercise, such as bending the arm and rotating the arm, to assess the patient's shoulder mobility.

3. Evaluate the patient's shoulder pain by asking about any specific symptoms or areas of pain, such as a lump or a clicking sound in the shoulder joint.

4. Assess the patient's ability to perform daily activities, such as brushing their teeth, using a cup, or turning a chair, to determine if the shoulder pain is impacting their daily functioning.

5. Ask the patient about any changes in their shoulder function or range of motion over the past few weeks, and consider any changes in their shoulder activities, such as changing jobs or engaging in physical activities that may cause shoulder pain.

6. Review the patient's medical history, including any previous shoulder surgeries, injuries, or medications, to determine if any of these factors may be contributing

In [47]:
from pprint import pprint

In [27]:
pprint(finetuned_output[finetuned_output.find('<|assistant|>'):])

('<|assistant|>\n'
 'Proper recovery is an essential aspect of powerlifting. It involves resting '
 'and recovering properly after a workout to ensure that the body can recover '
 'from the physical demands of lifting weights and prevent injury. Proper '
 'recovery helps to prevent muscle damage, reduce soreness, and improve '
 'performance. Proper recovery can also help to reduce the risk of injury and '
 'improve overall health and fitness. Powerlifters need to ensure that they '
 'are properly recovering after a workout to ensure that they can perform at '
 'their best in subsequent workouts. Proper recovery techniques include '
 'resting, stretching, and taking time off from lifting to allow the body to '
 'recover. It is also important to maintain a healthy diet and exercise '
 'regularly to ensure that the body is in a state of optimal recovery. Proper '
 'recovery is essential for maintaining peak performance and achieving '
 'successful powerlifting results.')


In [28]:
pprint(finetuned_output[base_output.find('<|assistant|>'):])

('<|assistant|>\n'
 'Proper recovery is an essential aspect of powerlifting. It involves resting '
 'and recovering properly after a workout to ensure that the body can recover '
 'from the physical demands of lifting weights and prevent injury. Proper '
 'recovery helps to prevent muscle damage, reduce soreness, and improve '
 'performance. Proper recovery can also help to reduce the risk of injury and '
 'improve overall health and fitness. Powerlifters need to ensure that they '
 'are properly recovering after a workout to ensure that they can perform at '
 'their best in subsequent workouts. Proper recovery techniques include '
 'resting, stretching, and taking time off from lifting to allow the body to '
 'recover. It is also important to maintain a healthy diet and exercise '
 'regularly to ensure that the body is in a state of optimal recovery. Proper '
 'recovery is essential for maintaining peak performance and achieving '
 'successful powerlifting results.')


In [12]:
from transformers import pipeline


In [13]:
new_model_name_ = new_model_name
generator = pipeline("text-generation", model=new_model_name_, tokenizer=tokenizer)


Device set to use cuda:0


In [14]:
print(generator("Forcing someone with hip anteversion to lift with a technique requires?", max_new_tokens=200)[0]['generated_text'])


Forcing someone with hip anteversion to lift with a technique requires?
Hip Anteversion? I need some advice on a hip anteversion exercise.
Hip Anteversion? I need some advice on a hip anteversion exercise.
Hip Anteposition? I need some advice on a hip anteposition exercise.
Hip Anteposition? I need some advice on a hip anteposition exercise.
Hip Anteversion? I need some advice on a hip anteversion exercise.
Hip Anteposition? I need some advice on a hip anteposition exercise.
Hip Anteversion? I need some advice on a hip anteversion exercise.
Hip Anteposition? I need some advice on a hip anteposition exercise.
Hip Anteversion? I need some advice on a hip anteversion exercise.
Hip Anteposition? I need some advice on a hip anteposition exercise.
Hip Anteversion? I need some advice on a hip anteversion exercise.
H


In [15]:
print(generator("What is the importance of proper recovery in powerlifting?", max_new_tokens=100)[0]['generated_text'])


What is the importance of proper recovery in powerlifting?


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Load the dataset from JSON file
dataset = load_dataset("json", data_files="qa_outputs_100_30072025.json")["train"]

# Merge 'instruction' and 'output' as prompt and target
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    return {"text": prompt}

dataset = dataset.map(format_example)

# Load tokenizer and model
model_id = "meta-llama/Llama-2-7b-hf"  # Must have access to this model from HuggingFace
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,  # Optional: enable quantization to save memory
    device_map="auto",
    trust_remote_code=True
)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # These depend on the model
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

# Training setup
training_args = TrainingArguments(
    output_dir="./lora-llama-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    fp16=True,
    save_total_limit=2,
    logging_steps=10,
    save_steps=500,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()


In [ ]:
####### WITH VALIDATION #######
from datasets import load_dataset

# Load full dataset (single JSON)
dataset = load_dataset("json", data_files=input_json_file_name)["train"]

# Split train/val
splits = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
val_dataset = splits["test"]

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Create Trainer with validation set
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train and evaluate
trainer.train()
eval_results = trainer.evaluate()
print(eval_results)
